In [ ]:
%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt

from d3pm_sc.mutual_info_schedule import get_a_b_func_cont, get_a_b_func_sc
from d3pm_sc import utils

import torch

N = 8
forward_kwargs = {"type":"gaussian",
                  "normalized": False,
                  "bandwidth":1/10}

In [ ]:
gamma = 0.95

L = utils.get_inf_gens(forward_kwargs, N)
rate = - (L.diagonal().min()) / (1-gamma) # L^* in sec 6.6 of the notes
K = L / rate + torch.eye(N)
# p0 = torch.randn(len(L)) ** 2
# p0 = p0 / p0.sum()
log_alpha, beta, mi = get_a_b_func_sc(K, p0)

ts = torch.linspace(0.0, 0.999, 1000) 
int_ts = -log_alpha(ts)
mis = mi(int_ts)
alphas = torch.exp(-int_ts)
log_betas = torch.log(beta(ts))
log_betas_norm = log_betas - log_betas.min()

plt.figure(figsize=[3, 3])
plt.plot(ts, mis.detach(), color='black')
plt.plot(ts, alphas.detach(), color='red')
plt.plot(ts, alphas.detach()**(1/rate), color='red', ls='--')
plt.plot(ts, log_betas_norm.detach() / log_betas_norm.max(), color='blue')
plt.ylabel("MI")
plt.xlabel("time")
plt.ylim(0, 1.1)
plt.xlim(0, 1)

fig, ax = plt.subplots(1, 3, figsize=[7, 2])
ax[0].semilogy(ts, int_ts, color='purple')
ax[0].semilogy(ts, int_ts/rate, color='purple', ls='--')
ax[1].semilogy(ts, alphas, color='red')
ax[1].semilogy(ts, alphas**(1/rate), color='red', ls='--')
ax[2].semilogy(ts, torch.exp(log_betas), color='blue')
ax[2].semilogy(ts, torch.exp(log_betas)/rate, color='blue', ls = '--')
plt.tight_layout()

In [ ]:
L = utils.get_inf_gens(forward_kwargs, N)
rate = - (L.diagonal().min()) / (1-gamma) # L^* in sec 6.6 of the notes
K = L / rate + torch.eye(N)
# p0 = torch.randn(len(L)) ** 2
# p0 = p0 / p0.sum()
log_alpha, beta, mi = get_a_b_func_cont(L, p0)

ts = torch.linspace(0.0, 0.999, 1000) 
int_ts = -log_alpha(ts)
mis = mi(int_ts)
alphas = torch.exp(-int_ts)
log_betas = torch.log(beta(ts))
log_betas_norm = log_betas - log_betas.min()

plt.figure(figsize=[3, 3])
plt.plot(ts, mis.detach(), color='black')
plt.plot(ts, alphas.detach(), color='red')
plt.plot(ts, log_betas_norm.detach() / log_betas_norm.max(), color='blue')
plt.ylabel("MI")
plt.xlabel("time")
plt.ylim(0, 1.1)
plt.xlim(0, 1)

fig, ax = plt.subplots(1, 3, figsize=[7, 2])
ax[0].semilogy(ts, int_ts, color='purple')
ax[1].semilogy(ts, alphas, color='red')
ax[2].semilogy(ts, torch.exp(log_betas), color='blue')
plt.tight_layout()

In [ ]:
%timeit int_ts = -log_alpha(ts)


In [ ]:
%timeit int_ts = beta(ts)
